Loop over a list of every item id. Make a dict for each id and have keys for the iqr range of each time avg. Also, add key for the mean of the 5m range.

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import time
from scipy.stats import iqr

In [2]:
headers = {'User-Agent':'GEoutlier-detection'}

In [3]:
api_endpt = "https://prices.runescape.wiki/api/v1/osrs"

In [4]:
timeseries_5m = '/timeseries?timestep=5m&id='
timeseries_1h = '/timeseries?timestep=1h&id='
timeseries_6h = '/timeseries?timestep=6h&id='

In [5]:
def get_series(api_endpt,timeseries,headers,item_id):
    items_response = requests.get((api_endpt+timeseries+str(item_id)), headers=headers)
    return items_response

In [6]:
def format_response(response):
    df = pd.DataFrame.from_dict(json.loads(response.text)['data'],orient='columns')
    return df

In [7]:
def formated_df(timeseries_pt,item_id):
    time_format = '%B-%d %Y %H:%M:%S'
    response = get_series(api_endpt,timeseries_pt,headers,item_id)
    df = format_response(response)
    df.timestamp = pd.to_datetime(df.timestamp,unit='s')
#     df.timestamp = df.timestamp.dt.strftime(time_format)
    return df

In [8]:
def iqr_calc(timeseries_data):
    interqr = iqr(timeseries_data.avgHighPrice,nan_policy='omit')
    iqrange = (timeseries_data.avgHighPrice.quantile()-(interqr/2),timeseries_data.avgHighPrice.quantile()+(interqr/2))

    return iqrange

In [9]:
import pickle
with open('unique_ids.pkl', 'rb') as f:
    unique_ids = pickle.load(f)

In [10]:
import time

In [11]:
#create item dict for stats
stats_dict = {}

In [12]:
from tqdm.notebook import tqdm_notebook

In [13]:
    
for item in tqdm_notebook(unique_ids):
    
    item_id = item
    
    #get the data
    data_5m = formated_df(timeseries_5m,item_id)
    data_1h = formated_df(timeseries_1h,item_id)
    data_6h = formated_df(timeseries_6h,item_id)

    data_list = [data_1h,data_6h]

    try:
        #calc the iqr
        iqrange_5m = iqr_calc(data_5m)
        iqrange_1h = iqr_calc(data_1h)
        iqrange_6h = iqr_calc(data_6h)
    except:
        continue
    #calc mean
    five_min_mean = data_5m.avgHighPrice.mean()

    #create dict for item_id
    stats_dict[f'{item_id}'] = {}

    #add stats
    stats_dict[f'{item_id}']['5m_range'] = iqrange_5m
    stats_dict[f'{item_id}']['1h_range'] = iqrange_1h
    stats_dict[f'{item_id}']['6h_range'] = iqrange_6h

    stats_dict[f'{item_id}']['5m_mean'] = five_min_mean

    time.sleep(1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [18]:
all_items_IQR = pd.DataFrame.from_dict(stats_dict).T

In [19]:
all_items_IQR.item_id = all_items_IQR.index 

<ipython-input-19-4d63de927cbf>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  all_items_IQR.item_id = all_items_IQR.index


In [21]:
all_items_IQR.to_csv('all_items_IQR.csv',index=False)